<a href="https://colab.research.google.com/github/YosriSaadi/Extraction_de_donn-es_via_OCR/blob/main/Extraction_de_donn%C3%A9es_via_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install easyocr keras-ocr img2table img2table[easyocr] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import easyocr
import json

def extract_text_from_image(image_path):
    # Initialize the reader with the languages you expect in the image
    reader = easyocr.Reader(['en', 'fr', 'it', 'es', 'la'])

    # Perform OCR on the image
    result = reader.readtext(image_path, detail=1)

    # Format the results
    formatted_results = []
    for (bbox, text, conf) in result:
        formatted_results.append({
            "text": text,
            "bounding_box": {
                "left": str(bbox[0][0]),
                "top": str(bbox[0][1]),
                "right": str(bbox[2][0]),
                "bottom": str(bbox[2][1])
            },
            "confidence": conf
        })

    return formatted_results

# Path to the image
image_path = '/content/Genova.png'

# Perform OCR and get the results
ocr_results = extract_text_from_image(image_path)

# Create the JSON output
output = {
    "extracted_texts": ocr_results
}

# Print the JSON output
print(json.dumps(output, indent=4))

# Optionally save to a file
with open('output.json', 'w') as f:
    json.dump(output, f, indent=4)

{
    "extracted_texts": [
        {
            "text": "Camaldoli",
            "bounding_box": {
                "left": "593",
                "top": "157",
                "right": "653",
                "bottom": "173"
            },
            "confidence": 0.9999339569631562
        },
        {
            "text": "G\u00eanes",
            "bounding_box": {
                "left": "284",
                "top": "218",
                "right": "360",
                "bottom": "248"
            },
            "confidence": 0.9999586830876289
        },
        {
            "text": "Bacino",
            "bounding_box": {
                "left": "234",
                "top": "320",
                "right": "269",
                "bottom": "334"
            },
            "confidence": 0.9309524431717044
        },
        {
            "text": "Pag",
            "bounding_box": {
                "left": "240",
                "top": "334",
                "right": "264",
        

In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm
from IPython.display import display_html
from PIL import Image as PILImage
from openpyxl import load_workbook
from io import BytesIO
import cv2
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
image_path = '/content/Genova.png'

In [ ]:
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
results = pipeline.recognize([image_path])

In [ ]:
pd.DataFrame(results[0], columns=['text', 'bbox'])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
keras_ocr.tools.drawAnnotations(plt.imread(image_path), results[0], ax=ax)
ax.set_title('Keras OCR Result Example')
plt.show()

##titre


In [ ]:
from img2table.document import Image

img_path = "t.png"

# Definition of image from path
img_from_path = Image(src=img_path)

# Definition of image from bytes
with open(img_path, 'rb') as f:
    img_bytes = f.read()
img_from_bytes = Image(src=img_bytes)

# Definition of image from file-like object
img_from_file_like = Image(src=BytesIO(img_bytes))

In [ ]:
# EasyOCR
from img2table.ocr import EasyOCR

easyocr = EasyOCR(lang=["en"], kw={"gpu": False})

In [ ]:
img = Image(src="t.png")

# Extract tables
extracted_tables = img.extract_tables()

In [ ]:
# Display extracted tables
table_img = cv2.imread("t.png")

for table in extracted_tables:
    for row in table.content.values():
        for cell in row:
            cv2.rectangle(table_img, (cell.bbox.x1, cell.bbox.y1), (cell.bbox.x2, cell.bbox.y2), (255, 0, 0), 2)

PILImage.fromarray(table_img)

In [ ]:
# Export to file
img.to_xlsx('tables.xlsx',
            ocr=easyocr,
            implicit_rows=False,
            borderless_tables=False,
            min_confidence=50)
